# Teaching notebook: Building a PopJSON larva model step by step

We develop the larva model incrementally in **three stages**:

1. Minimal Erlang-distributed development time (fixed).
2. Add temperature forcing to make development climate-sensitive.
3. Add a competing lifetime process and expose flows.

---

## Stage 1 — Minimal fixed Erlang development

Here we only model **development** with a fixed Erlang clock.


In [ ]:
# Example 1 PopJSON
cat(jsonlite::toJSON({
  "model": {
    "title": "Dynamically-structured population model",
    "type": "Population",
    "url": "https://github.com/kerguler/Population",
    "deterministic": true,
    "parameters": {
      "algorithm": "Population",
      "istep": 1e-13
    }
  },
  "populations": [
    {
      "id": "larva",
      "name": "The larva stage",
      "processes": [
        {
          "id": "larva_dev",
          "name": "Larva development time",
          "arbiter": "ACC_ERLANG",
          "value": [
            10,
            2
          ]
        }
      ]
    }
  ]
}, pretty=TRUE, auto_unbox=TRUE))


**Highlights**  
- One population (`larva`)  
- One process: `larva_dev` with `ACC_ERLANG` (mean=10, sd=2)  
- Minimal model header and deterministic integration settings  


---

## Stage 2 — Add temperature forcing

We introduce an **environmental driver** (`temp`), **functions** to map temperature to time, and **intermediates** to derive mean/sd for development.



In [ ]:
# Example 2 PopJSON
cat(jsonlite::toJSON({
  "model": {
    "title": "Dynamically-structured population model",
    "type": "Population",
    "url": "https://github.com/kerguler/Population",
    "deterministic": true,
    "parameters": {
      "algorithm": "Population",
      "istep": 1e-13
    }
  },
  "environ": [
    {
      "id": "temp",
      "name": "Temperature (in \u00b0C)"
    }
  ],
  "functions": {
    "briere1": [
      "define",
      [
        "T",
        "L",
        "R",
        "a"
      ],
      [
        "?",
        [
          "<=",
          "T",
          "L"
        ],
        "0.0",
        [
          "?",
          [
            ">=",
            "T",
            "R"
          ],
          "0.0",
          [
            "max",
            "0.0",
            [
              "*",
              "a",
              "T",
              [
                "-",
                "T",
                "L"
              ],
              [
                "sqrt",
                [
                  "-",
                  "R",
                  "T"
                ]
              ]
            ]
          ]
        ]
      ]
    ],
    "briere1_dev": [
      "define",
      [
        "T",
        "L",
        "R",
        "a"
      ],
      [
        "min",
        10000000000000.0,
        [
          "/",
          "1.0",
          [
            "briere1",
            "T",
            "L",
            "R",
            "a"
          ]
        ]
      ]
    ]
  },
  "parameters": [
    {
      "id": "param_dP_1",
      "constant": false,
      "name": "Larva development mean (minimum)",
      "value": 10.0
    },
    {
      "id": "param_dP_2",
      "constant": false,
      "name": "Larva development mean (maximum)",
      "value": 30.0
    },
    {
      "id": "param_dP_3",
      "constant": false,
      "name": "Larva development mean (strength)",
      "value": 0.0001
    }
  ],
  "intermediates": [
    {
      "id": "int_larva_dev_mean",
      "value": [
        "briere1_dev",
        [
          "temp",
          "TIME_1"
        ],
        "param_dP_1",
        "param_dP_2",
        "param_dP_3"
      ]
    },
    {
      "id": "int_larva_dev_stdev",
      "value": [
        "sqrt",
        "int_larva_dev_mean"
      ]
    }
  ],
  "populations": [
    {
      "id": "larva",
      "name": "The larva stage",
      "processes": [
        {
          "id": "larva_dev",
          "name": "Larva development time",
          "arbiter": "ACC_ERLANG",
          "value": [
            "int_larva_dev_mean",
            "int_larva_dev_stdev"
          ]
        }
      ]
    }
  ]
}, pretty=TRUE, auto_unbox=TRUE))


**Highlights**  
- `environ`: adds `temp`  
- `functions`: define thermal response (Brière type)  
- `parameters`: lower/upper bounds, scaling  
- `intermediates`: derive `int_larva_dev_mean`, `int_larva_dev_stdev` using yesterday's temp (`TIME_1`)  
- Process now uses these intermediates as its Erlang parameters  


---

## Stage 3 — Add lifetime process and transformations

We add a **second Erlang process** for lifetime (`larva_death`). Now larvae can exit by pupation *or* mortality. We also expose flows via `transformations`.



In [ ]:
# Example 3 PopJSON
cat(jsonlite::toJSON({
  "model": {
    "title": "Dynamically-structured population model",
    "type": "Population",
    "url": "https://github.com/kerguler/Population",
    "deterministic": true,
    "parameters": {
      "algorithm": "Population",
      "istep": 1e-13
    }
  },
  "environ": [
    {
      "id": "temp",
      "name": "Temperature (in \u00b0C)"
    }
  ],
  "functions": {
    "briere1": [
      "define",
      [
        "T",
        "L",
        "R",
        "a"
      ],
      [
        "?",
        [
          "<=",
          "T",
          "L"
        ],
        "0.0",
        [
          "?",
          [
            ">=",
            "T",
            "R"
          ],
          "0.0",
          [
            "max",
            "0.0",
            [
              "*",
              "a",
              "T",
              [
                "-",
                "T",
                "L"
              ],
              [
                "sqrt",
                [
                  "-",
                  "R",
                  "T"
                ]
              ]
            ]
          ]
        ]
      ]
    ],
    "briere1_dev": [
      "define",
      [
        "T",
        "L",
        "R",
        "a"
      ],
      [
        "min",
        10000000000000.0,
        [
          "/",
          "1.0",
          [
            "briere1",
            "T",
            "L",
            "R",
            "a"
          ]
        ]
      ]
    ]
  },
  "parameters": [
    {
      "id": "param_dP_1",
      "constant": false,
      "name": "Larva development mean (minimum)",
      "value": 10.0
    },
    {
      "id": "param_dP_2",
      "constant": false,
      "name": "Larva development mean (maximum)",
      "value": 30.0
    },
    {
      "id": "param_dP_3",
      "constant": false,
      "name": "Larva development mean (strength)",
      "value": 0.0001
    }
  ],
  "intermediates": [
    {
      "id": "int_larva_dev_mean",
      "value": [
        "briere1_dev",
        [
          "temp",
          "TIME_1"
        ],
        "param_dP_1",
        "param_dP_2",
        "param_dP_3"
      ]
    },
    {
      "id": "int_larva_dev_stdev",
      "value": [
        "sqrt",
        "int_larva_dev_mean"
      ]
    }
  ],
  "populations": [
    {
      "id": "larva",
      "name": "The larva stage",
      "processes": [
        {
          "id": "larva_death",
          "name": "Larva life time",
          "arbiter": "ACC_ERLANG",
          "value": [
            50.0,
            5.0
          ]
        },
        {
          "id": "larva_dev",
          "name": "Larva development time",
          "arbiter": "ACC_ERLANG",
          "value": [
            "int_larva_dev_mean",
            "int_larva_dev_stdev"
          ]
        }
      ]
    }
  ],
  "transformations": [
    {
      "id": "pupa",
      "value": "larva_dev"
    }
  ]
}, pretty=TRUE, auto_unbox=TRUE))


**Highlights**  
- Two processes: `larva_dev` (temperature-driven) and `larva_death` (fixed mean/sd)  
- Competing clocks → larvae pupate if development finishes first, die if lifetime ends first  
- `transformations`: expose pupation flow as an observable  
